In [12]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/d

In [13]:
def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, real_dim), name='input_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    return inputs_real, inputs_z

In [14]:
def generator(z, out_dim, n_units=128, reuse=False, alpha=0.01):
    """Generate image using whatever image input
    'Generate image using whatever image input'
    
    Args:
        z (int): Input dimension
        out_dim (int): output dimension
        n_units (int):  number of units
        reuse (boolean): reuse data or not
        alpha (float): Leaky ReLu coefficient
    
    Returns:
        tuple: output tensor as tuple of (out_dim) dimensions
    """
    with tf.variable_scope('generator', reuse=reuse):
        # first hidden layer using leaky Relu not to make the output of x < 0  to zero.
        h1 = tf.layers.dense(z, n_units, activation=None)
        h1 = tf.maximum(alpha*h1, h1)
        
        # second hidden layer
        # use tanh to transform output between -1 to 1 using Hyperbolic Tangent
        logits = tf.layers.dense(h1, out_dim, activation=None)
        out = tf.tanh(logits)
        
        return out
        

In [15]:
def discriminator(x, n_units=128, reuse=False, alpha=0.01):
    # Check
    with tf.variable_scope('discriminator', reuse=reuse):
        h1 = tf.layers.dense(x, n_units, activation=None)
        h1 = tf.maximum(alpha*h1, h1)
        logits = tf.layers.dense(h1, 1, activation=None)
        out = tf.sigmoid(logits)
        
        return out, logits

In [23]:
if __name__ == '__main__':
    # initialize hyper parameters (global variables)
    input_size = 784 # 28 * 28
    z_size = 100
    generator_hidden_size = 128
    discriminator_hidden_size = 128
    alpha = 0.01
    smooth = 0.1
    
    # define graph
    tf.reset_default_graph()
    input_real, input_z = model_inputs(input_size, z_size)
    
    generator_model = generator(input_z, input_size, n_units=generator_hidden_size, alpha=alpha)
    discriminator_model_real, discriminator_logits_real = discriminator(input_real, discriminator_hidden_size, False, alpha=alpha)
    discriminator_model_fake, discriminator_logits_fake = discriminator(generator_model, discriminator_hidden_size, True, alpha)
    
    # define loss functions
    discriminator_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=discriminator_logits_real,
        labels=tf.ones_like(discriminator_logits_real)*(1-smooth)
    ))
    discriminator_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=discriminator_logits_fake,
        labels=tf.zeros_like(discriminator_logits_real)
    ))
    discriminator_loss = discriminator_loss_real + discriminator_loss_fake
    generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=discriminator_logits_fake,
        labels=tf.ones_like(discriminator_logits_fake)
    ))